<a href="https://colab.research.google.com/github/MarinaVBsc/Project1/blob/main/proyecto_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Air Traffic Data Analysis").getOrCreate()

In [3]:
from google.colab import files
uploaded = files.upload()


Saving Air_Traffic_Passenger_Statistics.csv to Air_Traffic_Passenger_Statistics.csv


In [ ]:
df = spark.read.csv("Air_Traffic_Passenger_Statistics.csv", header=True, inferSchema=True)
airlines_count = df.select("Operating Airline").distinct().count()
print("Number of distinct operating airlines:", airlines_count)

In [ ]:
average_passengers = df.groupBy("Operating Airline").agg({"Passenger Count": "avg"})

import pandas as pd
import matplotlib.pyplot as plt

# Convertir el DataFrame de PySpark a Pandas
average_passengers_pd = average_passengers.toPandas()


# Configurar el gráfico y la tabla en Matplotlib
fig, ax = plt.subplots(figsize=(8, 2))  # Ajusta el tamaño de la figura
ax.axis('tight')
ax.axis('off')

# Crear la tabla en Matplotlib
table = ax.table(
    cellText=average_passengers_pd.values,
    colLabels=average_passengers_pd.columns,
    cellLoc='center',
    loc='center'
)

# Mejorar el formato de la tabla
table.auto_set_font_size(False)
table.set_fontsize(10)
table.auto_set_column_width(col=list(range(len(average_passengers_pd.columns))))

# Poner en negrita los encabezados de columna
for (i, j), cell in table.get_celld().items():
    if i == 0:  # Fila 0 corresponde a los encabezados
        cell.set_text_props(weight='bold')

# Mostrar la tabla
plt.show()


In [ ]:
# Importar funciones de PySpark
from pyspark.sql import functions as F

# Agrupar por 'GEO Region' y obtener el registro con mayor número de pasajeros
max_passenger_counts = df.groupBy('GEO Region') \
    .agg(F.max('Passenger Count').alias('Max Passenger Count'))

# Hacer un join y usar alias para evitar ambigüedad
deduplicated_df = max_passenger_counts \
    .join(df.alias("df2"), (max_passenger_counts['GEO Region'] == df.alias("df2")['GEO Region']) &
                (max_passenger_counts['Max Passenger Count'] == df.alias("df2")['Passenger Count'])) \
    .select("df2.Activity Period", "df2.Operating Airline", "df2.Operating Airline IATA Code", "df2.Published Airline", "df2.Published Airline IATA Code", "df2.GEO Summary", "df2.GEO Region", "df2.Activity Type Code", "df2.Price Category Code", "df2.Terminal", "df2.Boarding Area", "df2.Passenger Count", "df2.Adjusted Activity Type Code", "df2.Adjusted Passenger Count", "df2.Year", "df2.Month")  # Selecciona todas las columnas del DataFrame original explicitly usando el alias

# Convertir el DataFrame de PySpark a Pandas
deduplicated_df_pd = deduplicated_df.toPandas()


# Configurar el gráfico y la tabla en Matplotlib
fig, ax = plt.subplots(figsize=(8, 2))  # Ajusta el tamaño de la figura
ax.axis('tight')
ax.axis('off')

# Crear la tabla en Matplotlib
table = ax.table(
    cellText=deduplicated_df_pd.values,
    colLabels=deduplicated_df_pd.columns,
    cellLoc='center',
    loc='center'
)

# Mejorar el formato de la tabla
table.auto_set_font_size(False)
table.set_fontsize(10)
table.auto_set_column_width(col=list(range(len(deduplicated_df_pd.columns))))

# Poner en negrita los encabezados de columna
for (i, j), cell in table.get_celld().items():
    if i == 0:  # Fila 0 corresponde a los encabezados
        cell.set_text_props(weight='bold')

# Mostrar la tabla
plt.show()


In [7]:
# Volcar el DataFrame de la media de pasajeros por compañía en un CSV
average_passengers.write.csv('/average_passengers.csv', header=True)

# Volcar el DataFrame de registros únicos por 'GEO Region' en un CSV
deduplicated_df.write.csv('/deduplicated_passengers.csv', header=True)



In [ ]:
files.download('/average_passengers.csv')
files.download('/deduplicated_passengers.csv')

In [11]:
combined_df = average_passengers \
    .join(deduplicated_df.select('GEO Region', 'Passenger Count', 'Operating Airline'),
          average_passengers['Operating Airline'] == deduplicated_df['Operating Airline'],
          'outer') \
    .select(average_passengers['*'], deduplicated_df['GEO Region'], deduplicated_df['Passenger Count'])


In [9]:
# Volcar el nuevo DataFrame combinado en un solo CSV
combined_df.write.csv('/combined_results.csv', header=True)

In [ ]:
files.download('/combined_results.csv')

In [ ]:
stats_df = df.select(
    F.round(F.mean('Passenger Count'),2).alias('mean_passenger_count'),
    F.round(F.stddev('Passenger Count'),2).alias('stddev_passenger_count'),
    F.round(F.mean('Adjusted Passenger Count'),2).alias('mean_adjusted_passenger_count'),
    F.round(F.stddev('Adjusted Passenger Count'),2).alias('stddev_adjusted_passenger_count')
)
# Convertir el DataFrame de PySpark a Pandas
stats_pandas_df = stats_df.toPandas()

# Configurar el gráfico y la tabla en Matplotlib
fig, ax = plt.subplots(figsize=(8, 2))  # Ajusta el tamaño de la figura
ax.axis('tight')
ax.axis('off')

# Crear la tabla en Matplotlib
table = ax.table(
    cellText=stats_pandas_df.values,
    colLabels=stats_pandas_df.columns,
    cellLoc='center',
    loc='center'
)

# Mejorar el formato de la tabla
table.auto_set_font_size(False)
table.set_fontsize(10)
table.auto_set_column_width(col=list(range(len(stats_pandas_df.columns))))

# Poner en negrita los encabezados de columna
for (i, j), cell in table.get_celld().items():
    if i == 0:  # Fila 0 corresponde a los encabezados
        cell.set_text_props(weight='bold')

# Mostrar la tabla
plt.show()



In [ ]:
stats_df_airline = df.groupBy('Operating Airline').agg(
    F.round(F.mean('Passenger Count'),2).alias('mean_passenger_count'),
    F.round(F.stddev('Passenger Count'),2).alias('stddev_passenger_count'),
    F.round(F.mean('Adjusted Passenger Count'),2).alias('mean_adjusted_passenger_count'),
    F.round(F.stddev('Adjusted Passenger Count'),2).alias('stddev_adjusted_passenger_count')
)

# Convertir el DataFrame de PySpark a Pandas
stats_airline_pandas_df = stats_df_airline.toPandas()

# Configurar el gráfico y la tabla en Matplotlib
fig, ax = plt.subplots(figsize=(8, 2))  # Ajusta el tamaño de la figura
ax.axis('tight')
ax.axis('off')

# Crear la tabla en Matplotlib
table = ax.table(
    cellText=stats_airline_pandas_df.values,
    colLabels=stats_airline_pandas_df.columns,
    cellLoc='center',
    loc='center'
)

# Mejorar el formato de la tabla
table.auto_set_font_size(False)
table.set_fontsize(10)
table.auto_set_column_width(col=list(range(len(stats_airline_pandas_df.columns))))

# Mostrar la tabla
plt.show()



In [ ]:
sorted_airlines = stats_df_airline.orderBy('mean_passenger_count', ascending=False)

# Convertir el DataFrame de PySpark a Pandas
sorted_airlines_pd = sorted_airlines.toPandas()

# Configurar el gráfico y la tabla en Matplotlib
fig, ax = plt.subplots(figsize=(8, 2))  # Ajusta el tamaño de la figura
ax.axis('tight')
ax.axis('off')

# Crear la tabla en Matplotlib
table = ax.table(
    cellText=sorted_airlines_pd.values,
    colLabels=sorted_airlines_pd.columns,
    cellLoc='center',
    loc='center'
)

# Mejorar el formato de la tabla
table.auto_set_font_size(False)
table.set_fontsize(10)
table.auto_set_column_width(col=list(range(len(sorted_airlines_pd.columns))))

# Poner en negrita los encabezados de columna
for (i, j), cell in table.get_celld().items():
    if i == 0:  # Fila 0 corresponde a los encabezados
        cell.set_text_props(weight='bold')

# Mostrar la tabla
plt.show()


In [ ]:
sorted_airlines = stats_df_airline.orderBy('mean_passenger_count', ascending=True)
sorted_airlines.show()

In [ ]:
mean_passengers_per_year = df.groupBy('Operating Airline','Year').agg(F.round(F.mean('Passenger Count'),2).alias('mean_passengers'),
                                                                      F.round(F.stddev('Passenger Count'),2).alias('stddev_passengers'))
mean_passengers_per_year = mean_passengers_per_year.orderBy('mean_passengers', ascending=False)

# Convertir el DataFrame de PySpark a Pandas
mean_passengers_per_year_pd = mean_passengers_per_year.toPandas()

# Configurar el gráfico y la tabla en Matplotlib
fig, ax = plt.subplots(figsize=(8, 2))  # Ajusta el tamaño de la figura
ax.axis('tight')
ax.axis('off')

# Crear la tabla en Matplotlib
table = ax.table(
    cellText=mean_passengers_per_year_pd.values,
    colLabels=mean_passengers_per_year_pd.columns,
    cellLoc='center',
    loc='center'
)

# Mejorar el formato de la tabla
table.auto_set_font_size(False)
table.set_fontsize(10)
table.auto_set_column_width(col=list(range(len(mean_passengers_per_year_pd.columns))))

# Poner en negrita los encabezados de columna
for (i, j), cell in table.get_celld().items():
    if i == 0:  # Fila 0 corresponde a los encabezados
        cell.set_text_props(weight='bold')

# Mostrar la tabla
plt.show()


In [ ]:
top10_airlines_mean = mean_passengers_per_year.groupBy('Operating Airline').agg(F.mean('mean_passengers').alias('mean_passengers_global')).orderBy(F.desc('mean_passengers_global')).limit(10)

# Extraer los nombres de las aerolíneas en el top 10
top10_airline_names = [row['Operating Airline'] for row in top10_airlines_mean.collect()]

# Filtrar los datos para las aerolíneas top 10
top10_data = mean_passengers_per_year.filter(F.col('Operating Airline').isin(top10_airline_names))

# Convertir a Pandas para graficar
passengers_by_airline_year_pandas = top10_data.toPandas()

# Pivotar el DataFrame para tener las aerolíneas como columnas y los años como filas
pivot_df = passengers_by_airline_year_pandas.pivot(index='Year', columns='Operating Airline', values='mean_passengers')

import matplotlib.pyplot as plt

# Crear el gráfico de líneas
plt.figure(figsize=(10, 6))

# Graficar cada aerolínea
for airline in pivot_df.columns:
    plt.plot(pivot_df.index, pivot_df[airline], label=airline)

# Añadir títulos y etiquetas
plt.title('Top 10 Aerolíneas con la Media de Pasajeros más Alta a lo Largo de los Años')
plt.xlabel('Año')
plt.ylabel('Media de Pasajeros')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.grid(False)
plt.tight_layout()

# Mostrar el gráfico
plt.show()


In [18]:
from pyspark.ml.feature import StringIndexer

# Transformar columnas categóricas
indexers = [
    StringIndexer(inputCol=col, outputCol=col + "_Index").fit(df)
    for col in ["Price Category Code", "Operating Airline", "GEO Region", "Terminal", "Boarding Area", "Month"]
]
for indexer in indexers:
    df = indexer.transform(df)

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation

# Seleccionar las columnas indexadas y otras numéricas
assembler = VectorAssembler(inputCols=[
    "Passenger Count", "Adjusted Passenger Count",
    "Price Category Code_Index", "Operating Airline_Index",
    "GEO Region_Index", "Activity Period", "Year", "Month_Index",
    "Terminal_Index", "Boarding Area_Index",
], outputCol="features")

# Crear el vector de características
vector_df = assembler.transform(df).select("features")
correlation_matrix_all = Correlation.corr(vector_df, "features").head()[0]


In [ ]:
# Calcular la matriz de correlación

#Seleccion de columnas
selected_columns = ["Passenger Count", "Adjusted Passenger Count",
    "Price Category Code_Index", "Operating Airline_Index",
    "GEO Region_Index", "Activity Period", "Year", "Month",
    "Terminal_Index", "Boarding Area_Index"
]

corr_array = correlation_matrix_all.toArray()
corr_df = pd.DataFrame(data=corr_array, columns=selected_columns, index=selected_columns)

import seaborn as sns

# Configurar el tamaño del gráfico
plt.figure(figsize=(12, 8))

# Crear el mapa de calor
sns.heatmap(corr_df, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5, annot_kws={"size": 10})

# Ajustar título y etiquetas de los ejes
plt.title("Matriz de Correlación")
plt.xticks(rotation=45, ha="right", fontsize=10)
plt.yticks(fontsize=10)

# Mostrar el gráfico
plt.show()

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pandas.plotting import table


# Seleccionar las columnas predictoras
input_cols = [
    "Adjusted Passenger Count", "Price Category Code_Index",
    "Operating Airline_Index", "GEO Region_Index", "Activity Period",
    "Year", "Month_Index", "Terminal_Index", "Boarding Area_Index"
]
#Crear el VectorAssembler para la columna 'features'
assembler = VectorAssembler(inputCols=input_cols, outputCol="features")
data_with_features = assembler.transform(df).select("features", "Passenger Count")

# Configurar la regresión lineal
lr = LinearRegression(featuresCol="features", labelCol="Passenger Count")

# Entrenar el modelo
lr_model = lr.fit(data_with_features)

# Mostrar los resultados
coefficientes = lr_model.coefficients.toArray()
coeff_dict = dict(zip(input_cols, coefficientes))
intercept = lr_model.intercept

#Evaluar el modelo con un resumen
training_summary = lr_model.summary
rmse = training_summary.rootMeanSquaredError
r2 = training_summary.r2

#Crear Dataframe para coeficientes
coef_df = pd.DataFrame(list(coeff_dict.items()), columns=['Predictor', 'Coeficientes'])

#Agregar intercepto, RMSE y R2 al Dataframe

metrics_df = pd.DataFrame({
    'Metric': ['Intercepto', 'RMSE', 'R2'],
    'Valor': [intercept, rmse, r2]
})

#Mostrar coeficientes y metricas en tablas separadas
fig, axes = plt.subplots(2, 1, figsize=(10, 6))
axes[0].axis("tight")
axes[0].axis("off")
axes[1].axis("tight")
axes[1].axis("off")

# Convertir los DataFrames en tablas
table1 = table(axes[0], coef_df, loc="center", colWidths=[0.3, 0.3])
table2 = table(axes[1], metrics_df, loc="center", colWidths=[0.3, 0.3])

# Ajustar tamaño de fuente
table1.auto_set_font_size(False)
table1.set_fontsize(10)
table2.auto_set_font_size(False)
table2.set_fontsize(10)

# Mostrar la figura
plt.show()


